In [ ]:
#Try to run with GPU!
import os 
import pandas as pd
from matplotlib import image
import cv2

print(os.getcwd())
os.chdir('/kaggle/input/fiu-cap5610-spring22/')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

#Each image has dimensions (80,60,3)

In [ ]:
#This converts the word classes into numbers (we found just 4 classes)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(['Apparel', 'Accessories', 'Personal Care', 'Footwear'])

In [ ]:
# Transforms the labels to numbers

train_df['label'] = le.transform(train_df['label'])
train_df

In [ ]:
#This splits into train and validation sets (both have the labels)

from sklearn.model_selection import train_test_split

X_train, X_validation = train_test_split(train_df, test_size=0.3)
X_train

In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(X_train['label'],4)
validation_labels = to_categorical(X_validation['label'],4)
train_labels

In [ ]:
#imports for building the models
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, UpSampling2D, Dropout

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
def get_DenseNet(p=0.2):
    model = Sequential()
    #model.add(UpSampling2D(input_shape =(80,60,3), interpolation='bilinear'))
    base_model = DenseNet201(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=(80,60,3), pooling='max')
    #base_model.trainable = False
    model.add(base_model)
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(p+0.05))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(p-0.05))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(p-0.05))
    model.add(Dense(4, activation='softmax')) 
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [ ]:
# Get the model and print its architecture
p=0.4
model  = get_DenseNet(p)
model.summary()

In [ ]:
# Construct the training data and the validation data, may take time
import numpy as np
image_path ='/kaggle/input/fiu-cap5610-spring22/images/images/' #the images are here

def preprocess_image(image):
    
    return image/255.0

def build_matrix(id_lst):
    return np.array([preprocess_image(cv2.imread(image_path + str(img_id) +'.jpg')) for img_id in id_lst])

train_data = build_matrix(X_train['img_id'])
validation_data = build_matrix(X_validation['img_id'])
train_data.shape
#train_data

In [ ]:
print(train_data.shape, validation_data.shape)

In [ ]:
#callbacks, in this case just for the CNN
#early_stopping =  tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)
learning_rate = tf.keras.callbacks.ReduceLROnPlateau(patience = 5)
save_path = '/kaggle/working/DenseNet201_dropout_{}'.format(p)
check_point = tf.keras.callbacks.ModelCheckpoint(save_path ,save_best_only = True, monitor = 'val_accuracy')


In [ ]:
# Fit the model
model.fit(train_data, train_labels, validation_data = (validation_data, validation_labels) ,
          epochs=100, batch_size=32,callbacks = [learning_rate, check_point])

In [ ]:
loss, accuracy = model.evaluate(validation_data , validation_labels)
print('loss: ', loss , '\n', 'accuracy:', accuracy)

In [ ]:
load = True
if load:
    model_load = tf.keras.models.load_model(save_path)
    model = model_load

In [ ]:
test_data = build_matrix(test_df['img_id'])
predictions = model.predict(test_data) #Predicts probabilities
predictions_cl = np.argmax(predictions, axis = 1) #Takes the highest prediction to indicate the class in the one hot encoding
real_predictions = le.inverse_transform(predictions_cl) #Returns the labels as words
df_submission = test_df
df_submission['label'] = real_predictions
df_submission = df_submission[['img_id','label']]
df_submission 

In [ ]:
#Creates the csv file (change the name each time)
df_submission.to_csv('/kaggle/working/Dense201results.csv', index =False)  